### File for the keyboard data analysis in the lab-study

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import pingouin as pg
import matplotlib.pyplot as plt
from scipy.stats import sem
plt.style.use("seaborn-deep")

# SK Learn imports
from sklearn.pipeline import Pipeline
# from sklearn.model_selection import permutation_test_score
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeCV

# Imports for SK Learn RepeatedGroupKFold and Permutation Test Customizations
# imports to copy sk learn functionality and adapt it to the needs of our data
import numbers
from abc import ABCMeta, abstractmethod
import warnings
from inspect import signature
from sklearn.utils.validation import _num_samples, _deprecate_positional_args, check_array
from sklearn.base import _pprint

from joblib import Parallel, delayed
from sklearn.model_selection._split import check_cv
from sklearn.base import is_classifier, clone
from sklearn.utils import (indexable, check_random_state, _safe_indexing)
from sklearn.metrics import check_scoring
from sklearn.utils.metaestimators import _safe_split

In [ ]:
# import the keyboard feature dataset as a pandas dataframe (the dataset is in long format)
keyboard_data_long = pd.read_csv("Labstudy_Keyboard_Features.csv", sep="\t", encoding="utf-8", index_col=0)

# import the manipulation check variables dataset to complement the keyboard dataset
man_check_variables_data = pd.read_csv("Lab_Study_Manipulation_Check_Variables_for_ML.csv", sep="\t", encoding="utf-8",
                                       index_col=0)

# merge both datasets together into one dataset
keyboard_data_long = pd.concat([keyboard_data_long, man_check_variables_data], axis=1)

#### create a dataframe in wide format (for dependent t-tests and descriptive statistics)

In [ ]:

# get a name of the relevant columns for the wide data format
keyboard_column_names = [i for i in keyboard_data_long.columns if i != "par_Num" and i != "condition"
                         and i != "BPM" and i != "EDA" and i != "Valence" and i != "Arousal"]

# change the data from long to wide format
keyboard_data_wide = keyboard_data_long.pivot_table(index=['par_Num'], columns='condition', values=keyboard_column_names,
                                                    aggfunc='first')

# assign column names
keyboard_data_wide.columns = [f'{x}_{y}' for x, y in keyboard_data_wide.columns]


print("Shape of long format dataset:", keyboard_data_long.shape,
      "Shape of wide format dataset:", keyboard_data_wide.shape)

#### --- Compare the Keyboard Typing Features between the low-stress and high-stress condition ---

In [ ]:

# 1 Plot each variable pair (the same variable from the high-stress and low-stress condition) in one plot
for col in keyboard_data_wide.columns:
    if "HS" in col:
        # find the corresponding low_stress variable
        ls_string = col.replace("HS", "LS")
        # create a neutral variable name
        neutral_string = col.replace("HS", "")
        # plot both variables in one plot
        # side by side
        # plt.hist([keyboard_data_wide[col], keyboard_data_wide[ls_string]], label=["HS", "LS"])
        # on top of each other
        # plt.hist(keyboard_data_wide[col], alpha=0.5, label="HS")
        # plt.hist(keyboard_data_wide[ls_string], alpha=0.5, label="LS")
        # as seaborn distplots
        sns.distplot(keyboard_data_wide[col], hist=True, kde=True, kde_kws={"linewidth": 3}, label="HS")
        sns.distplot(keyboard_data_wide[ls_string], hist=True, kde=True, kde_kws={"linewidth": 3}, label="LS")
        plt.legend(loc="upper right")
        plt.title(neutral_string)
        plt.show()

In [ ]:
# 2 Get some descriptive stats about the datatset
desc_stats_wide_data = keyboard_data_wide.describe().sort_index(axis=1)

In [ ]:
# intitialize an empty dataframe to store the results of the paired sample t-test for each variable
paired_ttest_results = pd.DataFrame()

# 3 Run paired-sample T-Tests to compare each variable
for col in keyboard_data_wide.columns:
    if "HS" in col:
        # find the corresponding low_stress variable
        ls_string = col.replace("HS", "LS")
        # create a neutral variable name
        neutral_string = col.replace("HS", "")[:-1]
        # calculate the paired sample t-test
        paired_ttest = pg.ttest(keyboard_data_wide[col], keyboard_data_wide[ls_string], paired=True)
        # rename the pingouin index of the t-test dataframe to the variable name
        paired_ttest.index = [neutral_string]
        # concat the results of the variables t-test to the result dataframe
        paired_ttest_results = pd.concat([paired_ttest_results, paired_ttest])

#### --- Data Plotting before machine learning analysis ---

In [ ]:
# 1 Plot each Variable in a distplot
for col in keyboard_data_long.drop(["condition", "par_Num"], axis=1).columns:
    sns.distplot(keyboard_data_long[col], hist=True, kde=True, hist_kws={'edgecolor':'black'},
                 kde_kws={"linewidth": 3})
    plt.title(col)
    plt.show()

In [ ]:
# 2 Plot a pairplot with all relevant variables
sns.pairplot(keyboard_data_long.drop(["condition", "par_Num"], axis=1), corner=True)
plt.show()

In [ ]:
# 3 Plot a correlation matrix between the features

# Compute the correlation matrix
corr = keyboard_data_long.drop(["condition", "par_Num"], axis=1).corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0, square=True, linewidths=.5,
            cbar_kws={"shrink": .5}, annot=True)

plt.show()


### --Machine learning analysis--

#### --- Customize SK Learn code to implement repeated-group-k-fold cross validation ---

In [ ]:

# --- 1. Created a custom Repeated-Group-K-Fold Splitter that can be included in the Scikit-Learn Pipeline ---

# --- Copied from the Scikit Learn model_selection/_split.py
def _build_repr(self):
    # XXX This is copied from BaseEstimator's get_params
    cls = self.__class__
    init = getattr(cls.__init__, 'deprecated_original', cls.__init__)
    # Ignore varargs, kw and default values and pop self
    init_signature = signature(init)
    # Consider the constructor parameters excluding 'self'
    if init is object.__init__:
        args = []
    else:
        args = sorted([p.name for p in init_signature.parameters.values()
                       if p.name != 'self' and p.kind != p.VAR_KEYWORD])
    class_name = self.__class__.__name__
    params = dict()
    for key in args:
        # We need deprecation warnings to always be on in order to
        # catch deprecated param values.
        # This is set in utils/__init__.py but it gets overwritten
        # when running under python3 somehow.
        warnings.simplefilter("always", FutureWarning)
        try:
            with warnings.catch_warnings(record=True) as w:
                value = getattr(self, key, None)
                if value is None and hasattr(self, 'cvargs'):
                    value = self.cvargs.get(key, None)
            if len(w) and w[0].category == FutureWarning:
                # if the parameter is deprecated, don't show it
                continue
        finally:
            warnings.filters.pop(0)
        params[key] = value

    return '%s(%s)' % (class_name, _pprint(params, offset=len(class_name)))


class BaseCrossValidator(metaclass=ABCMeta):
    """Base class for all cross-validators
    Implementations must define `_iter_test_masks` or `_iter_test_indices`.
    """

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            The target variable for supervised learning problems.
        groups : array-like of shape (n_samples,), default=None
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        X, y, groups = indexable(X, y, groups)
        indices = np.arange(_num_samples(X))
        for test_index in self._iter_test_masks(X, y, groups):
            train_index = indices[np.logical_not(test_index)]
            test_index = indices[test_index]
            yield train_index, test_index

    # Since subclasses must implement either _iter_test_masks or
    # _iter_test_indices, neither can be abstract.
    def _iter_test_masks(self, X=None, y=None, groups=None):
        """Generates boolean masks corresponding to test sets.
        By default, delegates to _iter_test_indices(X, y, groups)
        """
        for test_index in self._iter_test_indices(X, y, groups):
            test_mask = np.zeros(_num_samples(X), dtype=np.bool)
            test_mask[test_index] = True
            yield test_mask

    def _iter_test_indices(self, X=None, y=None, groups=None):
        """Generates integer indices corresponding to test sets."""
        raise NotImplementedError

    @abstractmethod
    def get_n_splits(self, X=None, y=None, groups=None):
        """Returns the number of splitting iterations in the cross-validator"""

    def __repr__(self):
        return _build_repr(self)


class _BaseKFold(BaseCrossValidator, metaclass=ABCMeta):
    """Base class for KFold, GroupKFold, and StratifiedKFold"""

    @abstractmethod
    @_deprecate_positional_args
    def __init__(self, n_splits, *, shuffle, random_state):
        if not isinstance(n_splits, numbers.Integral):
            raise ValueError('The number of folds must be of Integral type. '
                             '%s of type %s was passed.'
                             % (n_splits, type(n_splits)))
        n_splits = int(n_splits)

        if n_splits <= 1:
            raise ValueError(
                "k-fold cross-validation requires at least one"
                " train/test split by setting n_splits=2 or more,"
                " got n_splits={0}.".format(n_splits))

        if not isinstance(shuffle, bool):
            raise TypeError("shuffle must be True or False;"
                            " got {0}".format(shuffle))

        if not shuffle and random_state is not None:  # None is the default
            # TODO 0.24: raise a ValueError instead of a warning
            warnings.warn(
                'Setting a random_state has no effect since shuffle is '
                'False. This will raise an error in 0.24. You should leave '
                'random_state to its default (None), or set shuffle=True.',
                FutureWarning
            )

        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,), default=None
            The target variable for supervised learning problems.
        groups : array-like of shape (n_samples,), default=None
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        if self.n_splits > n_samples:
            raise ValueError(
                ("Cannot have number of splits n_splits={0} greater"
                 " than the number of samples: n_samples={1}.")
                .format(self.n_splits, n_samples))

        for train, test in super().split(X, y, groups):
            yield train, test

    def get_n_splits(self, X=None, y=None, groups=None):
        """Returns the number of splitting iterations in the cross-validator
        Parameters
        ----------
        X : object
            Always ignored, exists for compatibility.
        y : object
            Always ignored, exists for compatibility.
        groups : object
            Always ignored, exists for compatibility.
        Returns
        -------
        n_splits : int
            Returns the number of splitting iterations in the cross-validator.
        """
        return self.n_splits


class _RepeatedSplits(metaclass=ABCMeta):
    """Repeated splits for an arbitrary randomized CV splitter.
    Repeats splits for cross-validators n times with different randomization
    in each repetition.
    Parameters
    ----------
    cv : callable
        Cross-validator class.
    n_repeats : int, default=10
        Number of times cross-validator needs to be repeated.
    random_state : int or RandomState instance, default=None
        Passes `random_state` to the arbitrary repeating cross validator.
        Pass an int for reproducible output across multiple function calls.
        See :term:`Glossary <random_state>`.
    **cvargs : additional params
        Constructor parameters for cv. Must not contain random_state
        and shuffle.
    """
    @_deprecate_positional_args
    def __init__(self, cv, *, n_repeats=10, random_state=None, **cvargs):
        if not isinstance(n_repeats, numbers.Integral):
            raise ValueError("Number of repetitions must be of Integral type.")

        if n_repeats <= 0:
            raise ValueError("Number of repetitions must be greater than 0.")

        if any(key in cvargs for key in ('random_state', 'shuffle')):
            raise ValueError(
                "cvargs must not contain random_state or shuffle.")

        self.cv = cv
        self.n_repeats = n_repeats
        self.random_state = random_state
        self.cvargs = cvargs

    def split(self, X, y=None, groups=None):
        """Generates indices to split data into training and test set.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of length n_samples
            The target variable for supervised learning problems.
        groups : array-like of shape (n_samples,), default=None
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        n_repeats = self.n_repeats
        rng = check_random_state(self.random_state)

        for idx in range(n_repeats):
            cv = self.cv(random_state=rng, shuffle=True,
                         **self.cvargs)
            for train_index, test_index in cv.split(X, y, groups):
                yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        """Returns the number of splitting iterations in the cross-validator
        Parameters
        ----------
        X : object
            Always ignored, exists for compatibility.
            ``np.zeros(n_samples)`` may be used as a placeholder.
        y : object
            Always ignored, exists for compatibility.
            ``np.zeros(n_samples)`` may be used as a placeholder.
        groups : array-like of shape (n_samples,), default=None
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Returns
        -------
        n_splits : int
            Returns the number of splitting iterations in the cross-validator.
        """
        rng = check_random_state(self.random_state)
        cv = self.cv(random_state=rng, shuffle=True,
                     **self.cvargs)
        return cv.get_n_splits(X, y, groups) * self.n_repeats

    def __repr__(self):
        return _build_repr(self)


# --- Adapted the GroupKFold class to be able to randomly shuffle the groups for repeated-group-k-fold-cross-validation
# The group indices are randomly shuffled and then split into train and test datasets -> The sk learn GroupKFold
# class does not allow random shuffling, because it splits based on the group sizes (which are equal in my dataset) ---
class RandomGroupKFold(_BaseKFold):

    def __init__(self, n_splits=5, *, shuffle=False,
                 random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def _iter_test_indices(self, X, y, groups):
        if groups is None:
            raise ValueError("The 'groups' parameter should not be None.")
        groups = check_array(groups, ensure_2d=False, dtype=None)

        unique_groups, groups = np.unique(groups, return_inverse=True)
        n_groups = len(unique_groups)

        if self.n_splits > n_groups:
            raise ValueError("Cannot have number of splits n_splits=%d greater"
                             " than the number of groups: %d."
                             % (self.n_splits, n_groups))

        # Weight groups by their number of occurrences
        n_samples_per_group = np.bincount(groups)

        # --- The code below is apated to randomly split the groups instead of splitting the groups based on group size
        # Distribute the most frequent groups first
        indices = np.arange(n_groups)

        if self.shuffle:
            check_random_state(self.random_state).shuffle(indices)

        # --- End of adapted code: Below is the original code from the sk learn GroupKFold class ---

        n_samples_per_group = n_samples_per_group[indices]

        # Total weight of each fold
        n_samples_per_fold = np.zeros(self.n_splits)

        # Mapping from group index to fold index
        group_to_fold = np.zeros(len(unique_groups))

        # Distribute samples by adding the largest weight to the lightest fold
        for group_index, weight in enumerate(n_samples_per_group):
            lightest_fold = np.argmin(n_samples_per_fold)
            n_samples_per_fold[lightest_fold] += weight
            group_to_fold[indices[group_index]] = lightest_fold

        indices = group_to_fold[groups]

        for f in range(self.n_splits):
            yield np.where(indices == f)[0]


# Added the class to be able to do Repeated GroupKFold Splitting: Copied and adapted the RepeatedKFold Class
class RepeatedGroupKFold(_RepeatedSplits):

    @_deprecate_positional_args
    def __init__(self, *, n_splits=5, n_repeats=10, random_state=None):
        super().__init__(
            RandomGroupKFold, n_repeats=n_repeats,
            random_state=random_state, n_splits=n_splits)


### --- Customize the sk learn permutation_test_score function ---

In [ ]:
# --- 2. Customize the SK Learn permutation_test_score function to return the list of all scores during cross validation
# instead of the mean of the list (as it is implemented in sk learn): The files are copy and pasted from the
# SK Learn model_selection/_validation.py file---

# SK Learn permutation Test score function adopted to the repeated group characteristics
@_deprecate_positional_args
def rep_group_permutation_test_score(estimator, X, y, *, groups=None, cv=None,
                           n_permutations=100, n_jobs=None, random_state=0,
                           verbose=0, scoring=None):
    """Evaluate the significance of a cross-validated score with permutations
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    estimator : estimator object implementing 'fit'
        The object to use to fit the data.
    X : array-like of shape at least 2D
        The data to fit.
    y : array-like of shape (n_samples,) or (n_samples, n_outputs) or None
        The target variable to try to predict in the case of
        supervised learning.
    groups : array-like of shape (n_samples,), default=None
        Labels to constrain permutation within groups, i.e. ``y`` values
        are permuted among samples with the same group identifier.
        When not specified, ``y`` values are permuted among all samples.
        When a grouped cross-validator is used, the group labels are
        also passed on to the ``split`` method of the cross-validator. The
        cross-validator uses them for grouping the samples  while splitting
        the dataset into train/test set.
    scoring : str or callable, default=None
        A single str (see :ref:`scoring_parameter`) or a callable
        (see :ref:`scoring`) to evaluate the predictions on the test set.
        If None the estimator's score method is used.
    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
        - None, to use the default 5-fold cross validation,
        - int, to specify the number of folds in a `(Stratified)KFold`,
        - :term:`CV splitter`,
        - An iterable yielding (train, test) splits as arrays of indices.
        For int/None inputs, if the estimator is a classifier and ``y`` is
        either binary or multiclass, :class:`StratifiedKFold` is used. In all
        other cases, :class:`KFold` is used.
        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validation strategies that can be used here.
        .. versionchanged:: 0.22
            ``cv`` default value if None changed from 3-fold to 5-fold.
    n_permutations : int, default=100
        Number of times to permute ``y``.
    n_jobs : int, default=None
        The number of CPUs to use to do the computation.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.
    random_state : int, RandomState instance or None, default=0
        Pass an int for reproducible output for permutation of
        ``y`` values among samples. See :term:`Glossary <random_state>`.
    verbose : int, default=0
        The verbosity level.
    Returns
    -------
    score : float
        The true score without permuting targets.
    permutation_scores : array of shape (n_permutations,)
        The scores obtained for each permutations.
    pvalue : float
        The p-value, which approximates the probability that the score would
        be obtained by chance. This is calculated as:
        `(C + 1) / (n_permutations + 1)`
        Where C is the number of permutations whose score >= the true score.
        The best possible p-value is 1/(n_permutations + 1), the worst is 1.0.
    Notes
    -----
    This function implements Test 1 in:
        Ojala and Garriga. Permutation Tests for Studying Classifier
        Performance.  The Journal of Machine Learning Research (2010)
        vol. 11
        `[pdf] <http://www.jmlr.org/papers/volume11/ojala10a/ojala10a.pdf>`_.
    """
    X, y, groups = indexable(X, y, groups)

    cv = check_cv(cv, y, classifier=is_classifier(estimator))
    scorer = check_scoring(estimator, scoring=scoring)
    random_state = check_random_state(random_state)

    # We clone the estimator to make sure that all the folds are
    # independent, and that it is pickle-able.
    # returned scores are list of scores instead of the mean score
    scores = _permutation_test_score(clone(estimator), X, y, groups, cv, scorer)
    permutation_scores = Parallel(n_jobs=n_jobs, verbose=verbose)(
        delayed(_permutation_test_score)(
            clone(estimator), X, _shuffle(y, groups, random_state),
            groups, cv, scorer)
        for _ in range(n_permutations))
    #  in repeated 5-fold cross validation, it is a 25 * n_permutations matrix --> To get the get "original" permutation
    # p-value, we need to take the mean on axis 1 to get mean(25 cv scores) * n_permutations
    permutation_scores = np.array(permutation_scores)
    pvalue = (np.sum(np.mean(permutation_scores, axis=1) >= np.mean(scores)) + 1.0) / (n_permutations + 1)
    return scores, permutation_scores, pvalue


# Changed this function to return the list of scores instead of the mean of the list of scores
def _permutation_test_score(estimator, X, y, groups, cv, scorer):
    """Auxiliary function for permutation_test_score"""
    scores = []
    for train, test in cv.split(X, y, groups):
        X_train, y_train = _safe_split(estimator, X, y, train)
        X_test, y_test = _safe_split(estimator, X, y, test, train)
        estimator.fit(X_train, y_train)
        scores.append(scorer(estimator, X_test, y_test))
    return scores


# this function is unchanged
def _shuffle(y, groups, random_state):
    """Return a shuffled copy of y eventually shuffle among same groups."""
    if groups is None:
        indices = random_state.permutation(len(y))
    else:
        indices = np.arange(len(groups))
        for group in np.unique(groups):
            this_mask = (groups == group)
            indices[this_mask] = random_state.permutation(indices[this_mask])
    return _safe_indexing(y, indices)

#### --- Helper Function to carry out the machine learning analysis ---

In [ ]:

# plot the results of the permutation procedure
def plot_permutation_test_results(permutation_scores, model_scores, pvalue, p_tresh, method, plot_title):

    # set the default colors to the seaborn color codes
    sns.set_color_codes()

    # initialize a figure with two subplots (one above the other with a 15%, 85% ratio)
    f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)})

    # plot a boxplot of the
    sns.boxplot(model_scores, color="LightYellow", showmeans=True, meanline=True,
                meanprops={"linestyle": "-", "linewidth": "3", "color": "DarkKhaki"},
                medianprops={"linestyle": "None", "linewidth": "0"},
                flierprops={"marker": "o", "markerfacecolor": "LightYellow"},
                ax=ax_box)

    # set the x_label of the Boxplot to None
    ax_box.set_xlabel("")
    # Remove the borders of the "graph" around the boxplot
    ax_box.axis("off")

    # plot a histogram of the permutation scores
    sns.histplot(permutation_scores, color="b", bins=15, ax=ax_hist, label="Permutation Scores")

    # plot a vertical line of the mean cv score
    ax_hist.axvline(np.mean(model_scores), color="DarkKhaki", linewidth=3,
                    label="Model Score: %s (%s), \np = %s" % (
                    np.round(np.mean(model_scores), 2), np.round(np.std(model_scores), 2), pvalue))
    # plot a vertical line of the mean permutation score
    ax_hist.axvline(np.mean(permutation_scores), color="Navy", linewidth=3,
               label="Permutation Mean: " "%s" % (np.round(np.mean(permutation_scores), 2)))
    # plot a vertical line of the significance threshold
    ax_hist.axvline(p_tresh,
             label="Sig. Threshold: " "%s" % p_tresh, color="darkgreen", linestyle="--", linewidth=3)

    # remove the top and left corner of the
    sns.despine(top=True, right=True, left=False, bottom=False)

    # set title
    ax_box.set(title=plot_title)
    # set the axis labels (depends on the score used for classification or regression)
    if method == "Classification":
        score_label = "Accuracy Score"
    else:
        score_label = "R² Score"
    plt.xlabel(score_label)
    plt.ylabel("Frequency")
    # loc legend to the top left
    plt.legend(loc="upper left")
    plt.savefig(plot_title + ".png")
    plt.show()
    plt.close()


# calculates a k-fold cross val score and compares the mean performance score with a distribution of n models that
# were trained with permutated class labels
# more information can be found here:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.permutation_test_score.html
def ml_permutation_test(iv, dv, algorithm, groups, method, cv_repeats, permutation_repeats, procedure_title):

    # save the results of the permutation test
    results = {}

    # initiate the pipeline and select the desired configurations for handling multicollinearity and
    # the feature selection procedure
    # make a pipeline that does the preprocessing and outputs the cross validation score
    pipeline = Pipeline([
        ("standardization", StandardScaler()),
        ("clf", algorithm)
    ])

    # initialize the repeated k-fold iterator with non-overlapping groups
    cv = RepeatedGroupKFold(n_splits=5, n_repeats=cv_repeats)
    # cv = GroupKFold(n_splits=5)

    # get the scoring function
    if method == "Classification":
        scorer = "accuracy"
    else:
        scorer = "r2"

    # get the cross validation scores, the standard deviation of the cv scores, the permuation scores and the
    # p-value of the permutation test
    scores, permutation_scores, pvalue = rep_group_permutation_test_score(
        pipeline, iv, dv, scoring=scorer, cv=cv, groups=groups, n_permutations=permutation_repeats)

    # get the statistical values of the model scores
    model_score = np.mean(scores)
    model_std = np.std(scores)
    model_std_err = sem(scores)

    # get the statistical values of the permutation (in repeated cv, its n_splits * n_repeats scores for each
    # permutation run --> num of permutation_scores = n_splits * n_repeats scores * n_permutations)
    perm_score = np.mean(permutation_scores)
    perm_std = np.std(np.mean(permutation_scores, axis=1))
    # Get the significance threshold (the classification model must be better than 95% of the permutated models)
    sig_tresh = np.percentile(np.mean(permutation_scores, axis=1), 95.0)

    # save the results
    results["Mean_score"] = np.round(model_score, 2)
    results["SD_score"] = np.round(model_std, 2)
    results["SE_score"] = np.round(model_std_err, 2)
    results["p_value"] = np.round(pvalue, 5)
    results["Mean_Permutation_score"] = np.round(perm_score, 2)
    results["Std_Permutation_score"] = np.round(perm_std, 2)
    results["Sig_Treshold"] = sig_tresh

    plot_permutation_test_results(permutation_scores=np.mean(permutation_scores, axis=1),
                                  model_scores=scores,
                                  pvalue=np.round(pvalue, 3),
                                  p_tresh=np.round(sig_tresh, 2),
                                  method=method,
                                  plot_title=procedure_title)

    # output the results and scores
    return results, scores


# helper function for repeated k-fold cross validation without a permutation test
def rep_kfold_cv(iv, dv, algorithm, groups, method, cv_repeats):

    # save the results of the permutation test
    results = {}

    # initiate the pipeline and select the desired configurations for handling multicollinearity and
    # the feature selection procedure
    # make a pipeline that does the preprocessing and outputs the cross validation score
    pipeline = Pipeline([
        ("standardization", StandardScaler()),
        ("clf", algorithm)
    ])

    # initialize the repeated k-fold iterator with non-overlapping groups
    cv = RepeatedGroupKFold(n_splits=5, n_repeats=cv_repeats)

    # get the scoring function
    if method == "Classification":
        scorer = "accuracy"
    else:
        scorer = "r2"

    # get the scores of repeated 5-fold cross validation
    scores = cross_val_score(pipeline, X=iv, y=dv, groups=groups, scoring=scorer, cv=cv)

    # calculate and save the results
    results["Mean_score"] = np.round(np.mean(scores), 2)
    results["SD_score"] = np.round(np.std(scores), 2)
    results["SE_score"] = np.round(sem(scores), 2)

    return results

#### --- Helper function to run the machine learning analysis with the selected settings ---

In [ ]:
# Run the permutation test with the "final settings" and save the results in a pandas dataframe
def get_ml_results(dataset, method, permutation_test, dv):

    # get a list of all keyboard features (machine learning predictors)
    keyboard_features = [i for i in dataset.columns if i != "par_Num" and i != "condition"
                         and i != "BPM" and i != "EDA" and i != "Valence" and i != "Arousal"]

    ml_results = {}

    # dict with different algorithms for classification/regression: add or remove algorithms here
    # we chose a simple neighbors classifier and a lasso regression with cv search for the best alpha
    algorithms = {
        "Classification": {
            "kNN_class": KNeighborsClassifier(n_neighbors=3)},
        "Regression": {
            "RidgeReg": RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])}

    }

    # drop rows if the column of the dependent variable contains NaN
    dataset = dataset.dropna(subset=[dv])

    # get the dependent and independent variable of the machine learning analysis
    predictors = dataset.loc[:, keyboard_features]
    dependent_variable = dataset[dv]
    if method == "Classification":
        dependent_variable = dependent_variable.replace(["HS", "LS"], [0., 1.])

    # perform the permuation test with all algorithms
    for alg in algorithms[method]:
        if permutation_test:
            analysis_title = method + " Permutation test with target " + dv + " using " + alg
            print(analysis_title)
            # get the results of the permutation test
            result_dic, model_scores = ml_permutation_test(iv=predictors,
                                                           dv=dependent_variable,
                                                           algorithm=algorithms[method][alg],
                                                           groups=dataset["par_Num"],
                                                           method=method,
                                                           cv_repeats=5,
                                                           permutation_repeats=1000,
                                                           procedure_title=analysis_title)
        else:
            analysis_title = method + " Repeated cv with target " + dv + " using " + alg
            print(analysis_title)
            result_dic = rep_kfold_cv(iv=predictors,
                                      dv=dependent_variable,
                                      algorithm=algorithms[method][alg],
                                      groups=dataset["par_Num"],
                                      method=method,
                                      cv_repeats=5)
            print(result_dic)

        # save the results with labeled index in the results dictionary
        ml_results[(method, dv, alg)] = result_dic

    # create a multiindexed dataframe from the results dictionary
    permutation_results_df = pd.DataFrame(ml_results).T

    return permutation_results_df

#### --- Do the machine learning analysis ---

Use the raw keyboard feature data

##### 1 Condition Classification

In [ ]:

# 1 Condition Classification
cond_classification_results = get_ml_results(keyboard_data_long, method="Classification", dv="condition",
                                             permutation_test=True)

# add an index key to the results dataframe
cond_classification_results = pd.concat([cond_classification_results], keys=["Raw_Data"])

##### 2 Regression on Valence, Arousal, BPM and EDA

In [ ]:
# Valence
valence_regression_results = get_ml_results(keyboard_data_long, method="Regression", dv="Valence",
                                            permutation_test=False)

In [ ]:
# BPM
bpm_regression_results = get_ml_results(keyboard_data_long, method="Regression", dv="BPM",
                                        permutation_test=False)

In [ ]:
# EDA
eda_regression_results = get_ml_results(keyboard_data_long, method="Regression", dv="EDA",
                                        permutation_test=False)

In [ ]:
# merge the regression results into one dataframe and add a raw data index
merged_results = pd.concat([pd.concat([valence_regression_results, arousal_regression_results,
                            bpm_regression_results, eda_regression_results])], keys=["Raw_Data"])


#### -- create a difference score dataset to do the same analysis with the difference score data ---

In [ ]:

# 1 Create difference score dataset
def create_difference_score_data(dataset):

    # calculate the difference scores (substract rows from each other)
    diff_part1 = dataset.drop("condition", axis=1).groupby("par_Num").diff()
    diff_part2 = dataset.drop("condition", axis=1).groupby("par_Num").diff(periods=-1)
    # "Merge both dataframes together to replace the missing values with the difference scores
    merged_diff = diff_part1.fillna(diff_part2)
    # add the remaining variables to the dataframe
    merged_diff = pd.concat([merged_diff, dataset.loc[:, ["condition", "par_Num"]]], axis=1)
    print(diff_part1.shape, diff_part2.shape, merged_diff.shape)

    return merged_diff


diff_dataset = create_difference_score_data(keyboard_data_long)

##### Condition Classification using the difference score data

In [ ]:
diff_cond_classification_results = get_ml_results(diff_dataset, method="Classification", dv="condition",
                                                  permutation_test=True)

# add a raw data index to the results dataframe
diff_cond_classification_results = pd.concat([diff_cond_classification_results], keys=["Diff_Data"])


##### Regression on valence, arousal. BPM and EDA using the difference score data

In [ ]:
# Valence
diff_valence_regression_results = get_ml_results(diff_dataset, method="Regression", dv="Valence",
                                                 permutation_test=False)

In [ ]:
# Arousal
diff_arousal_regression_results = get_ml_results(diff_dataset, method="Regression", dv="Arousal",
                                                 permutation_test=False)


In [ ]:
# BPM
diff_bpm_regression_results = get_ml_results(diff_dataset, method="Regression", dv="BPM",
                                             permutation_test=False)

In [ ]:
# merge the regression results into one dataframe and add a raw data index
diff_merged_results = pd.concat([pd.concat([diff_valence_regression_results,
                                 diff_arousal_regression_results, diff_bpm_regression_results,
                                 diff_eda_regression_results])], keys=["Diff_Data"])

#### --- Save the results of all data analysis in one excel file ---

In [ ]:
# Merge the results and save all data analysis results in one excel file
ml_regression_results = pd.concat([merged_results, diff_merged_results])
ml_classification_results = pd.concat([cond_classification_results, diff_cond_classification_results])

# save the dataframe plus the descriptive stats as an excel file
with pd.ExcelWriter("Labstudy_Keyboard_Feature_Analysis_Results.xlsx") as writer:
    desc_stats_wide_data.to_excel(writer, float_format="%.3f", sheet_name="Descriptive_Stats")
    paired_ttest_results.to_excel(writer, float_format="%.4f", sheet_name="t-test_results")
    ml_classification_results.to_excel(writer, float_format="%.4f", sheet_name="Machine_Learning_Classification")
    ml_regression_results.to_excel(writer, float_format="%.4f", sheet_name="Machine_Learning_regression")
